In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [2]:
# Assign %pwd to working_dir
working_dir = %pwd

In [4]:
# Reading kmeans output From Disk- start with the 13 clusters we discovered from clustering our PCA results
spotify = pd.read_csv((working_dir)+"/Data/spotify-classlabels-kmeans.csv", delimiter = ",")

##### Split data into training and testing

In [5]:
target_labels = spotify['cluster']
spotify = spotify.drop(columns=['cluster'])

spotify

,artists,id,name,year,mode_0,acousticness,explicit_1,instrumentalness,key,valence
0,['Mamie Smith'],0cS0A1fUEUd1EW3FcF8AEI,Keep A Song In Your Soul,1920s,1,0.991000,0,0.000522,0.454545,0.6340
1,"[""Screamin' Jay Hawkins""]",0hbkKFIJm7Z05H8Zl9w30f,I Put A Spell On You,1920s,1,0.643000,0,0.026400,0.454545,0.9500
2,['Mamie Smith'],11m7laMUgmOKqI3oYzuhne,Golfing Papa,1920s,0,0.993000,0,0.000018,0.000000,0.6890
3,['Oscar Velazquez'],19Lc5SfJJ5O1oaxY0fpwfh,True House Music - Xavier Santos & Carlos Gomi...,1920s,0,0.000173,0,0.801000,0.181818,0.0422
4,['Mixe'],2hJjbsLCytGsnAHfdsLejp,Xuniverxe,1920s,1,0.295000,1,0.000246,0.909091,0.2990
...,...,...,...,...,...,...,...,...,...,...
174384,"['DJ Combo', 'Sander-7', 'Tony T']",46LhBf6TvYjZU2SMvGZAbn,The One,2020s,1,0.009170,0,0.000060,0.545455,0.1860
174385,['Alessia Cara'],7tue2Wemjd0FZzRtDrQFZd,A Little More,2020s,0,0.795000,0,0.000000,0.363636,0.2280
174386,['Roger Fly'],48Qj61hOdYmUCFJbpQ29Ob,Together,2020s,1,0.806000,0,0.920000,0.363636,0.7140
174387,['Taylor Swift'],1gcyHQpBQ1lfXGdhZmWrHP,champagne problems,2020s,0,0.920000,1,0.000000,0.000000,0.3200


In [6]:

train, test, target_train, target_test = train_test_split(spotify, target_labels, test_size=0.2, random_state=33)

In [7]:
pca_names = np.array(
    ['mode_0', 
     'acousticness', 
     'explicit_1', 
     'instrumentalness', 
     'key', 
     'valence'])
train_numeric = train[pca_names]

test_numeric = test[pca_names]

In [8]:
# going to implement grid search but needed to look at other homeworks! Returning to this! 